In [ ]:
import pandas as pd

df= pd.read_csv("E:/Python ML/binance.csv")  

print("First few rows of the dataset:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nMissing Values:")
print(df.isnull().sum())

print("\nBasic Statistics:")
print(df.describe())


First few rows of the dataset:
              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None

Missing Values:
Port_IDs         0
Trade_History    1
dtype: int64

Basic Statistics:
           Port_IDs
count  1.500000e+02
mean   3.975655e+18
std    6.887470e+16
min    3.672

In [ ]:
import ast

df = df.dropna(subset=["Trade_History"]).reset_index(drop=True)

# Parse Trade_History
def parse_trade_history(trade_history):
    try:
        return ast.literal_eval(trade_history) 
    except (ValueError, SyntaxError):
        return []

df["Parsed_History"] = df["Trade_History"].apply(parse_trade_history)

exploded_df = df.explode("Parsed_History").reset_index(drop=True)

# Expand the nested dictionary into columns
trade_details = exploded_df["Parsed_History"].apply(pd.Series)

# Combine expanded columns with account information
final_df = pd.concat([exploded_df[["Port_IDs"]], trade_details], axis=1)

print("Expanded Trade Details:")
print(final_df.head())


Expanded Trade Details:
              Port_IDs           time    symbol side      price       fee  \
0  3925368433214965504  1718899656000   SOLUSDT  BUY  132.53700 -0.994027   
1  3925368433214965504  1718899618000  DOGEUSDT  BUY    0.12182 -0.279796   
2  3925368433214965504  1718899618000  DOGEUSDT  BUY    0.12182 -0.039494   
3  3925368433214965504  1718899616000  DOGEUSDT  BUY    0.12182 -0.008284   
4  3925368433214965504  1718899616000  DOGEUSDT  BUY    0.12182 -0.046109   

  feeAsset    quantity quantityAsset  realizedProfit realizedProfitAsset  \
0     USDT  1988.05500          USDT             0.0                USDT   
1     USDT  1398.98088          USDT             0.0                USDT   
2     USDT   197.47022          USDT             0.0                USDT   
3     USDT    16.56752          USDT             0.0                USDT   
4     USDT    92.21774          USDT             0.0                USDT   

  baseAsset      qty positionSide  activeBuy  
0       S

In [4]:
# Ensure realizedProfit and quantity are numeric
final_df["realizedProfit"] = pd.to_numeric(final_df["realizedProfit"], errors="coerce")
final_df["quantity"] = pd.to_numeric(final_df["quantity"], errors="coerce")

# Calculate metrics for each Port_ID
metrics_df = final_df.groupby("Port_IDs").agg(
    Initial_Investment=("quantity", lambda x: x[final_df["side"] == "BUY"].sum()),
    Net_Profit=("realizedProfit", "sum"),
    Total_Positions=("realizedProfit", "count"),
    Win_Positions=("realizedProfit", lambda x: (x > 0).sum())
).reset_index()

# Calculate additional metrics
metrics_df["ROI"] = (metrics_df["Net_Profit"] / metrics_df["Initial_Investment"]) * 100
metrics_df["Win_Rate"] = (metrics_df["Win_Positions"] / metrics_df["Total_Positions"]) * 100

# Display the calculated metrics
print("Metrics for Each Account:")
print(metrics_df.head())


Metrics for Each Account:
              Port_IDs  Initial_Investment   Net_Profit  Total_Positions  \
0  3672754654734989568        61735.947451   566.597660              474   
1  3733192481840423936       580774.260700  2923.977200              689   
2  3768170840939476993          766.603827   243.668899               14   
3  3784403294629753856       377877.824072  2521.814305             6050   
4  3786761687746711808        32724.336700   205.021400               82   

   Win_Positions        ROI   Win_Rate  
0          210.0   0.917776  44.303797  
1          553.0   0.503462  80.261248  
2            6.0  31.785505  42.857143  
3         1829.0   0.667362  30.231405  
4           37.0   0.626510  45.121951  


In [5]:
# Sharpe Ratio: Mean return / Std deviation of returns
def calculate_sharpe_ratio(trades):
    returns = trades["realizedProfit"]
    if returns.std() == 0:
        return 0  # Avoid division by zero
    return returns.mean() / returns.std()

# MDD: Maximum Drawdown calculation
def calculate_mdd(trades):
    cumulative_returns = trades["realizedProfit"].cumsum()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    return drawdown.min()  # Largest negative drawdown

# Calculate Sharpe Ratio and MDD for each account
sharpe_mdd_df = final_df.groupby("Port_IDs").apply(lambda trades: pd.Series({
    "Sharpe_Ratio": calculate_sharpe_ratio(trades),
    "MDD": calculate_mdd(trades)
})).reset_index()

# Merge metrics with Sharpe Ratio and MDD
metrics_df = pd.merge(metrics_df, sharpe_mdd_df, on="Port_IDs", how="left")

# Display the updated metrics
print("Updated Metrics with Sharpe Ratio and MDD:")
print(metrics_df.head())


Updated Metrics with Sharpe Ratio and MDD:
              Port_IDs  Initial_Investment   Net_Profit  Total_Positions  \
0  3672754654734989568        61735.947451   566.597660              474   
1  3733192481840423936       580774.260700  2923.977200              689   
2  3768170840939476993          766.603827   243.668899               14   
3  3784403294629753856       377877.824072  2521.814305             6050   
4  3786761687746711808        32724.336700   205.021400               82   

   Win_Positions        ROI   Win_Rate  Sharpe_Ratio       MDD  
0          210.0   0.917776  44.303797      0.185274 -0.460781  
1          553.0   0.503462  80.261248      0.060265 -0.760355  
2            6.0  31.785505  42.857143      0.424277  0.000000  
3         1829.0   0.667362  30.231405      0.106585 -0.177261  
4           37.0   0.626510  45.121951      0.215150 -2.318212  


In [ ]:
#  weights for each metric
weights = {
    "ROI": 0.3, #Since the main objective of trading is profitability, ROI (0.3) is assigned the most weight.
    "Win_Rate": 0.25, #Sharpe Ratio (0.25) and Win Rate (0.25) are equally weighted as they both evaluate the caliber and reliability of performance.
    "Sharpe_Ratio": 0.25,
    "MDD": -0.2  # Negative weight for lower MDD is better
}

# Normalize metrics for comparison
metrics_df["ROI_Score"] = metrics_df["ROI"] / metrics_df["ROI"].max()
metrics_df["Win_Rate_Score"] = metrics_df["Win_Rate"] / metrics_df["Win_Rate"].max()
metrics_df["Sharpe_Score"] = metrics_df["Sharpe_Ratio"] / metrics_df["Sharpe_Ratio"].max()
metrics_df["MDD_Score"] = metrics_df["MDD"] / metrics_df["MDD"].min()  # Invert MDD for scoring

# Calculate weighted scores
metrics_df["Overall_Score"] = (
    metrics_df["ROI_Score"] * weights["ROI"] +
    metrics_df["Win_Rate_Score"] * weights["Win_Rate"] +
    metrics_df["Sharpe_Score"] * weights["Sharpe_Ratio"] +
    metrics_df["MDD_Score"] * weights["MDD"]
)

# Rank accounts
metrics_df["Rank"] = metrics_df["Overall_Score"].rank(ascending=False)

# Sort by rank and get the top 20
top_20_accounts = metrics_df.sort_values("Rank").head(20)

# Save results to CSV
top_20_accounts.to_csv("Top_20_Accounts.csv", index=False)

# Display top 20 accounts
print("Top 20 Accounts:")
print(top_20_accounts[["Port_IDs", "Overall_Score", "Rank"]])


Top 20 Accounts:
                Port_IDs  Overall_Score  Rank
8    3826087012661391104       0.597490   1.0
2    3768170840939476993       0.590609   2.0
47   3956048468100538880       0.442256   3.0
16   3891020560590657281       0.398761   4.0
143  4039129759104249600       0.393143   5.0
62   3986814617275053313       0.381685   6.0
100  4022641794255717633       0.376403   7.0
144  4039279455324236544       0.362076   8.0
133  4035430878731345664       0.361160   9.0
39   3944658614777849089       0.356749  10.0
36   3943533600390906881       0.349149  11.0
92   4017110277719148289       0.331605  12.0
14   3886752488982104320       0.331307  13.0
109  4029422834086627072       0.330323  14.0
112  4029749871687083265       0.326399  15.0
58   3977234346014419201       0.317444  16.0
148  4041860229502600193       0.316949  17.0
88   4013955990069013504       0.303612  18.0
74   3998659472131949824       0.300006  19.0
17   3907081197088384000       0.294405  20.0
